In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
def data_preprocessing_binary(path):
  df = pd.read_csv(path)
  df_encoded = pd.get_dummies(df[['Proto','State']], columns=['Proto', 'State'])

  select_features_df = df[[ 'AckDat', 'sHops', 'Seq','TcpRtt', 'dMeanPktSz', 'Offset', 'sTtl',  'Mean', 'SrcTCPBase', 'sMeanPktSz', 'DstLoss', 'Loss', 'dTtl', 'SrcBytes', 'TotBytes']]
  select_features_df['sHops'].fillna(df['sHops'].mean(), inplace = True)
  select_features_df['sTtl'].fillna(df['sTtl'].mean(), inplace = True)
  select_features_df['SrcTCPBase'].fillna(df['SrcTCPBase'].mean(), inplace = True)
  select_features_df['dTtl'].fillna(df['dTtl'].mean(), inplace = True)
  df_merged = pd.concat([select_features_df, df_encoded], axis=1)

  df_output_encoded = df['Label']
  df_output_encoded_1 = df_output_encoded.replace(['Benign', 'Malicious'],
                        [0, 1], inplace=False)


  #returns input and output dataframe
  return df_merged, df_output_encoded_1



In [3]:
def standardize(input_df):
  scaler = StandardScaler()
  df_standardized = scaler.fit_transform(input_df)
  df_standardized = pd.DataFrame(df_standardized, columns=input_df.columns)
  return df_standardized


In [4]:
# create rnn binary model method.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Import necessary libraries
from sklearn.base import BaseEstimator, TransformerMixin

# Define a custom transformer to incorporate RNN model into the pipeline
class RNNWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, rnn_model):
        self.rnn_model = rnn_model

    def fit(self, X, y=None):
        self.rnn_model.fit(X, y, epochs=10, batch_size=32)  # Train the RNN model
        return self

    def transform(self, X):
        return self.rnn_model.predict(X)


In [6]:
#Method calls for Binary RNN model....
path = '/content/drive/MyDrive/5G NIDD/Combined.csv'
input, output = data_preprocessing_binary(path)

In [7]:
#uses the standardize methods for EDA.
input_standardized = standardize(input)

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming X_train and X_test are your input features
X = input_standardized.values  # Convert DataFrame to numpy array
y = output.values  # Convert DataFrame to numpy array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for the RNN model
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [9]:
# Define and compile the RNN model
rnn_model = create_rnn_model(input_shape=(1, X_train.shape[1]))
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_rnn, y_test))


Epoch 1/10
30398/30398 [==============================] - 119s 4ms/step - loss: 0.0126 - accuracy: 0.9968 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 2/10
30398/30398 [==============================] - 115s 4ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.0015 - val_accuracy: 0.9996
Epoch 3/10
30398/30398 [==============================] - 105s 3ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 0.0012 - val_accuracy: 0.9997
Epoch 4/10
30398/30398 [==============================] - 113s 4ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0016 - val_accuracy: 0.9994
Epoch 5/10
30398/30398 [==============================] - 109s 4ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 6/10
30398/30398 [==============================] - 114s 4ms/step - loss: 0.0010 - accuracy: 0.9996 - val_loss: 0.0012 - val_accuracy: 0.9994
Epoch 7/10
30398/30398 [==============================] - 117s 4ms/step - loss: 9.3530e-04 - accuracy: 0.9996 - 

In [10]:
# Predict on test data using the trained RNN model
y_pred = rnn_model.predict(X_test_rnn)

# Threshold the predictions
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Print the first few binary predictions
# print("Sample of binary predictions:")
# print(y_pred_binary[:10])


7600/7600 [==============================] - 14s 2ms/step


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the predictions
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

# Print the evaluation metrics
print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Evaluation Metrics:
Accuracy: 0.9997080327990197
Precision: 0.9999321265432727
Recall: 0.9995861151821093
F1 Score: 0.9997590909245139
